Курсовая работа

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
plt.style.use('fivethirtyeight')

%matplotlib inline

In [2]:
data=pd.read_csv('Course_work/train.csv')
test=pd.read_csv('Course_work/test.csv')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 20 columns):
Id               10000 non-null int64
DistrictId       10000 non-null int64
Rooms            10000 non-null float64
Square           10000 non-null float64
LifeSquare       7887 non-null float64
KitchenSquare    10000 non-null float64
Floor            10000 non-null int64
HouseFloor       10000 non-null float64
HouseYear        10000 non-null int64
Ecology_1        10000 non-null float64
Ecology_2        10000 non-null object
Ecology_3        10000 non-null object
Social_1         10000 non-null int64
Social_2         10000 non-null int64
Social_3         10000 non-null int64
Healthcare_1     5202 non-null float64
Helthcare_2      10000 non-null int64
Shops_1          10000 non-null int64
Shops_2          10000 non-null object
Price            10000 non-null float64
dtypes: float64(8), int64(9), object(3)
memory usage: 1.5+ MB


In [4]:
data.describe()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Price
count,10000.00000,10000.000000,10000.000000,10000.000000,7887.000000,10000.000000,10000.000000,10000.000000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,5202.000000,10000.000000,10000.000000,10000.000000
mean,8383.40770,50.400800,1.890500,56.315775,37.199645,6.273300,8.526700,12.609400,3.990166e+03,0.118858,24.687000,5352.157400,8.039200,1142.904460,1.319500,4.231300,214138.857399
std,4859.01902,43.587592,0.839512,21.058732,86.241209,28.560917,5.241148,6.775974,2.005003e+05,0.119025,17.532614,4006.799803,23.831875,1021.517264,1.493601,4.806341,92872.293865
min,0.00000,0.000000,0.000000,1.136859,0.370619,0.000000,1.000000,0.000000,1.910000e+03,0.000000,0.000000,168.000000,0.000000,0.000000,0.000000,0.000000,59174.778028
25%,4169.50000,20.000000,1.000000,41.774881,22.769832,1.000000,4.000000,9.000000,1.974000e+03,0.017647,6.000000,1564.000000,0.000000,350.000000,0.000000,1.000000,153872.633942
50%,8394.50000,36.000000,2.000000,52.513310,32.781260,6.000000,7.000000,13.000000,1.977000e+03,0.075424,25.000000,5285.000000,2.000000,900.000000,1.000000,3.000000,192269.644879
75%,12592.50000,75.000000,2.000000,65.900625,45.128803,9.000000,12.000000,17.000000,2.001000e+03,0.195781,36.000000,7227.000000,5.000000,1548.000000,2.000000,6.000000,249135.462171
max,16798.00000,209.000000,19.000000,641.065193,7480.592129,2014.000000,42.000000,117.000000,2.005201e+07,0.521867,74.000000,19083.000000,141.000000,4849.000000,6.000000,23.000000,633233.466570


In [5]:
data.loc[data['Rooms']==0,'Rooms']=1

In [6]:
data.loc[data['Rooms']==10.0,'Rooms']=2
data.loc[data['Rooms']==19.0,'Rooms']=1

In [7]:
def prepare_square(df):
    df.loc[df['Square']>(df['Rooms']*40+40),'Square']=df['Rooms']*40+40
    df.loc[df['Square']<(df['Rooms']*15),'Square']=df['Rooms']*15
    return df

In [8]:
data=prepare_square(data)

In [9]:
data

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
0,14038,35,2.0,47.981561,29.442751,6.0,7,9.0,1969,8.903972e-02,B,B,33,7976,5,NaN,0,11,B,184966.930730
1,15053,41,3.0,65.683640,40.049543,8.0,7,9.0,1978,6.998930e-05,B,B,46,10309,1,240.0,1,16,B,300009.450063
2,4765,53,2.0,44.947953,29.197612,0.0,8,12.0,1968,4.963726e-02,B,B,34,7759,0,229.0,1,3,B,220925.908524
3,5809,58,2.0,53.352981,52.731512,9.0,8,17.0,1977,4.378852e-01,B,B,23,5735,3,1084.0,0,5,B,175616.227217
4,10783,99,1.0,39.649192,23.776169,7.0,11,12.0,1976,1.233889e-02,B,B,35,5776,1,2078.0,2,4,B,150226.531644
5,12915,59,3.0,80.384479,46.683720,12.0,5,17.0,2011,3.094791e-01,B,B,35,7715,4,990.0,0,6,B,215898.447742
6,14549,154,2.0,62.254114,37.160377,7.0,3,5.0,1960,4.605564e-01,B,B,20,4386,14,NaN,1,5,B,296021.204377
7,11993,74,2.0,80.312926,NaN,0.0,14,0.0,1977,7.577876e-02,B,B,6,1437,3,NaN,0,2,B,221244.156664
8,5172,1,2.0,64.511437,NaN,1.0,9,17.0,1977,7.122317e-03,B,B,1,264,0,NaN,0,1,B,229102.795999
9,8649,23,1.0,46.461409,18.915552,8.0,13,17.0,2014,7.577876e-02,B,B,6,1437,3,NaN,0,2,B,95380.220993


In [10]:
data.describe()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Price
count,10000.00000,10000.000000,10000.000000,10000.000000,7887.000000,10000.000000,10000.000000,10000.000000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,5202.000000,10000.000000,10000.000000,10000.000000
mean,8383.40770,50.400800,1.887900,56.079022,37.199645,6.273300,8.526700,12.609400,3.990166e+03,0.118858,24.687000,5352.157400,8.039200,1142.904460,1.319500,4.231300,214138.857399
std,4859.01902,43.587592,0.812527,18.629121,86.241209,28.560917,5.241148,6.775974,2.005003e+05,0.119025,17.532614,4006.799803,23.831875,1021.517264,1.493601,4.806341,92872.293865
min,0.00000,0.000000,1.000000,15.000000,0.370619,0.000000,1.000000,0.000000,1.910000e+03,0.000000,0.000000,168.000000,0.000000,0.000000,0.000000,0.000000,59174.778028
25%,4169.50000,20.000000,1.000000,41.784790,22.769832,1.000000,4.000000,9.000000,1.974000e+03,0.017647,6.000000,1564.000000,0.000000,350.000000,0.000000,1.000000,153872.633942
50%,8394.50000,36.000000,2.000000,52.535436,32.781260,6.000000,7.000000,13.000000,1.977000e+03,0.075424,25.000000,5285.000000,2.000000,900.000000,1.000000,3.000000,192269.644879
75%,12592.50000,75.000000,2.000000,65.925548,45.128803,9.000000,12.000000,17.000000,2.001000e+03,0.195781,36.000000,7227.000000,5.000000,1548.000000,2.000000,6.000000,249135.462171
max,16798.00000,209.000000,6.000000,240.000000,7480.592129,2014.000000,42.000000,117.000000,2.005201e+07,0.521867,74.000000,19083.000000,141.000000,4849.000000,6.000000,23.000000,633233.466570


In [11]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 19 columns):
Id               5000 non-null int64
DistrictId       5000 non-null int64
Rooms            5000 non-null float64
Square           5000 non-null float64
LifeSquare       3959 non-null float64
KitchenSquare    5000 non-null float64
Floor            5000 non-null int64
HouseFloor       5000 non-null float64
HouseYear        5000 non-null int64
Ecology_1        5000 non-null float64
Ecology_2        5000 non-null object
Ecology_3        5000 non-null object
Social_1         5000 non-null int64
Social_2         5000 non-null int64
Social_3         5000 non-null int64
Healthcare_1     2623 non-null float64
Helthcare_2      5000 non-null int64
Shops_1          5000 non-null int64
Shops_2          5000 non-null object
dtypes: float64(7), int64(9), object(3)
memory usage: 742.3+ KB


In [12]:
test.describe()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1
count,5000.000000,5000.000000,5000.000000,5000.000000,3959.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,2623.000000,5000.00000,5000.000000
mean,8412.595400,51.279200,1.910000,56.449500,36.158810,5.976800,8.632000,12.601000,1984.392600,0.119874,24.933800,5406.900000,8.262600,1146.657263,1.31940,4.242800
std,4832.674037,44.179466,0.838594,19.092787,17.825287,9.950018,5.483228,6.789213,18.573149,0.120070,17.532202,4026.614773,23.863762,1044.744231,1.47994,4.777365
min,1.000000,0.000000,0.000000,1.378543,0.333490,0.000000,1.000000,0.000000,1908.000000,0.000000,0.000000,168.000000,0.000000,0.000000,0.00000,0.000000
25%,4221.750000,21.000000,1.000000,41.906231,23.092026,1.000000,4.000000,9.000000,1973.000000,0.019509,6.000000,1564.000000,0.000000,325.000000,0.00000,1.000000
50%,8320.500000,37.000000,2.000000,52.921340,32.925087,6.000000,7.000000,12.000000,1977.000000,0.072158,25.000000,5285.000000,2.000000,900.000000,1.00000,3.000000
75%,12598.250000,77.000000,2.000000,66.285129,45.174091,9.000000,12.000000,17.000000,2000.000000,0.195781,36.000000,7287.000000,5.000000,1548.000000,2.00000,6.000000
max,16795.000000,212.000000,17.000000,223.453689,303.071094,620.000000,78.000000,99.000000,2020.000000,0.521867,74.000000,19083.000000,141.000000,4849.000000,6.00000,23.000000


In [13]:
test.loc[test['Rooms']==0,'Rooms']=1

In [14]:
test.loc[test['Rooms']==10.0,'Rooms']=2
test.loc[test['Rooms']==19.0,'Rooms']=1

In [15]:
test=prepare_square(test)

In [16]:
test.describe()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1
count,5000.000000,5000.000000,5000.000000,5000.000000,3959.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,2623.000000,5000.00000,5000.000000
mean,8412.595400,51.279200,1.910400,56.409960,36.158810,5.976800,8.632000,12.601000,1984.392600,0.119874,24.933800,5406.900000,8.262600,1146.657263,1.31940,4.242800
std,4832.674037,44.179466,0.837921,18.898287,17.825287,9.950018,5.483228,6.789213,18.573149,0.120070,17.532202,4026.614773,23.863762,1044.744231,1.47994,4.777365
min,1.000000,0.000000,1.000000,15.000000,0.333490,0.000000,1.000000,0.000000,1908.000000,0.000000,0.000000,168.000000,0.000000,0.000000,0.00000,0.000000
25%,4221.750000,21.000000,1.000000,41.920449,23.092026,1.000000,4.000000,9.000000,1973.000000,0.019509,6.000000,1564.000000,0.000000,325.000000,0.00000,1.000000
50%,8320.500000,37.000000,2.000000,52.928839,32.925087,6.000000,7.000000,12.000000,1977.000000,0.072158,25.000000,5285.000000,2.000000,900.000000,1.00000,3.000000
75%,12598.250000,77.000000,2.000000,66.295632,45.174091,9.000000,12.000000,17.000000,2000.000000,0.195781,36.000000,7287.000000,5.000000,1548.000000,2.00000,6.000000
max,16795.000000,212.000000,17.000000,255.000000,303.071094,620.000000,78.000000,99.000000,2020.000000,0.521867,74.000000,19083.000000,141.000000,4849.000000,6.00000,23.000000


In [17]:
train, valid =train_test_split(data, test_size=0.3, random_state=42)

In [18]:
train.shape, valid.shape

((7000, 20), (3000, 20))

In [19]:
distr_size=(train['DistrictId'].value_counts()>180).astype(int).reset_index().rename(columns={'index':'DistrictId',\
                                                                                   'DistrictId':'large_district'})
distr_size

,DistrictId,large_district
0,27,1
1,1,1
2,23,1
3,6,1
4,9,1
5,62,0
6,30,0
7,53,0
8,58,0
9,21,0


In [20]:
def add_distr_size(df, distr_size):
    df=pd.merge(df, distr_size, on='DistrictId', how='left')
    df['large_district']=df['large_district'].fillna(0)
    return df

In [21]:
train = add_distr_size(train, distr_size)
train

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,...,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price,large_district
0,14604,23,1.0,41.681380,22.796166,8.0,14,17.0,2015,0.075779,...,B,6,1437,3,NaN,0,2,B,88504.384965,1
1,5621,23,3.0,160.000000,161.504222,12.0,5,3.0,1977,0.014073,...,B,2,475,0,NaN,0,0,B,207007.956663,1
2,235,87,1.0,39.710131,19.538663,8.0,4,17.0,1986,0.100456,...,B,43,7227,0,NaN,1,6,A,182126.280899,0
3,16258,48,3.0,96.056784,98.152802,1.0,15,1.0,2017,0.041125,...,B,46,9515,5,NaN,1,10,B,524365.550705,0
4,10773,77,3.0,79.195825,44.421062,10.0,16,17.0,1984,0.298205,...,B,16,4048,3,NaN,1,3,B,322048.433990,0
5,3766,62,3.0,127.417560,NaN,0.0,9,0.0,1977,0.072158,...,B,2,629,1,NaN,0,0,A,282877.332154,0
6,11463,129,3.0,72.953912,39.478723,9.0,16,15.0,1991,0.460556,...,B,20,4386,14,NaN,1,5,B,358129.914474,0
7,15037,11,2.0,46.430852,31.678912,5.0,2,5.0,1960,0.038693,...,B,28,6533,1,1015.0,2,5,B,175973.694767,0
8,4389,39,2.0,54.326263,34.394391,9.0,3,12.0,1974,0.428826,...,B,25,5011,8,730.0,0,2,B,240019.792399,0
9,15829,104,3.0,75.777599,45.381156,9.0,2,9.0,1985,0.327018,...,B,12,2270,23,2500.0,1,0,B,468039.075500,0


In [22]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7000 entries, 0 to 6999
Data columns (total 21 columns):
Id                7000 non-null int64
DistrictId        7000 non-null int64
Rooms             7000 non-null float64
Square            7000 non-null float64
LifeSquare        5514 non-null float64
KitchenSquare     7000 non-null float64
Floor             7000 non-null int64
HouseFloor        7000 non-null float64
HouseYear         7000 non-null int64
Ecology_1         7000 non-null float64
Ecology_2         7000 non-null object
Ecology_3         7000 non-null object
Social_1          7000 non-null int64
Social_2          7000 non-null int64
Social_3          7000 non-null int64
Healthcare_1      3642 non-null float64
Helthcare_2       7000 non-null int64
Shops_1           7000 non-null int64
Shops_2           7000 non-null object
Price             7000 non-null float64
large_district    7000 non-null int32
dtypes: float64(8), int32(1), int64(9), object(3)
memory usage: 1.1+ MB


In [23]:
valid = add_distr_size(valid, distr_size)
valid

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,...,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price,large_district
0,3702,21,2.0,48.812195,34.679779,5.0,5,5.0,1963,0.034331,...,B,25,5487,3,NaN,0,4,B,181530.459031,0.0
1,12848,61,3.0,81.103039,49.310278,8.0,4,4.0,1960,0.298205,...,B,16,4048,3,NaN,1,3,B,260456.004692,0.0
2,2239,6,3.0,82.882978,3.977650,1.0,8,17.0,1977,0.243205,...,B,5,1564,0,540.0,0,0,B,219945.304640,1.0
3,15611,23,1.0,33.863324,29.993297,0.0,5,4.0,1977,0.034656,...,B,0,168,0,NaN,0,0,B,66883.280318,1.0
4,5634,52,1.0,43.095135,NaN,1.0,10,17.0,1977,0.371149,...,B,34,7065,1,750.0,2,5,B,114086.065201,0.0
5,14726,53,3.0,84.569998,60.613297,12.0,18,24.0,2017,0.049637,...,B,34,7759,0,229.0,1,3,B,359065.287230,0.0
6,7247,69,2.0,54.663924,29.266790,9.0,13,17.0,1999,0.000991,...,B,33,6783,3,NaN,1,9,B,377445.078933,0.0
7,2018,94,1.0,33.521641,30.155171,1.0,17,16.0,2015,0.282798,...,B,33,8667,2,NaN,0,6,B,381912.162109,0.0
8,14587,6,1.0,40.696294,42.182049,1.0,4,17.0,2014,0.243205,...,B,5,1564,0,540.0,0,0,B,133072.279471,1.0
9,3643,46,2.0,44.369058,31.064926,5.0,1,5.0,1965,0.188784,...,B,31,6137,4,NaN,0,1,B,184036.005146,0.0


In [24]:
valid.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3000 entries, 0 to 2999
Data columns (total 21 columns):
Id                3000 non-null int64
DistrictId        3000 non-null int64
Rooms             3000 non-null float64
Square            3000 non-null float64
LifeSquare        2373 non-null float64
KitchenSquare     3000 non-null float64
Floor             3000 non-null int64
HouseFloor        3000 non-null float64
HouseYear         3000 non-null int64
Ecology_1         3000 non-null float64
Ecology_2         3000 non-null object
Ecology_3         3000 non-null object
Social_1          3000 non-null int64
Social_2          3000 non-null int64
Social_3          3000 non-null int64
Healthcare_1      1560 non-null float64
Helthcare_2       3000 non-null int64
Shops_1           3000 non-null int64
Shops_2           3000 non-null object
Price             3000 non-null float64
large_district    3000 non-null float64
dtypes: float64(9), int64(9), object(3)
memory usage: 515.6+ KB


In [25]:
from sklearn.ensemble import RandomForestRegressor as RF

In [26]:
model =RF(n_estimators=50, max_depth=8, min_samples_leaf=5, random_state=42)

In [27]:
feats= ['Rooms','Square','large_district']

In [28]:
model.fit(train.loc[:,feats], train['Price'])

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=8,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=5, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=None,
           oob_score=False, random_state=42, verbose=0, warm_start=False)

In [29]:
pred_train =model.predict(train.loc[:,feats])

In [30]:
pred_train

array([125156.09915661, 303467.76580109, 178730.11438339, ...,
       236062.83643917, 223196.98682705, 317147.80555137])

In [31]:
pred_train.shape

(7000,)

In [32]:
pred_valid =model.predict(valid.loc[:,feats])

In [33]:
pred_valid

array([203313.80160061, 342732.47014261, 227317.84045984, ...,
       232587.30939075, 181050.08028466, 237709.57246849])

In [34]:
from sklearn.metrics import r2_score as r2

In [35]:
r2(train['Price'], pred_train)

0.6017098036215645

In [36]:
r2(valid['Price'], pred_valid)

0.541075519158956

In [37]:
test.shape

(5000, 19)

In [38]:
test = add_distr_size(test, distr_size)
test

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,large_district
0,725,58,2.0,49.882643,33.432782,6.0,6,14.0,1972,0.310199,B,B,11,2748,1,NaN,0,0,B,0.0
1,15856,74,2.0,69.263183,NaN,1.0,6,1.0,1977,0.075779,B,B,6,1437,3,NaN,0,2,B,0.0
2,5480,190,1.0,15.000000,15.948246,12.0,2,5.0,1909,0.000000,B,B,30,7538,87,4702.0,5,5,B,0.0
3,15664,47,2.0,73.046609,51.940842,9.0,22,22.0,2007,0.101872,B,B,23,4583,3,NaN,3,3,B,0.0
4,14275,27,1.0,47.527111,43.387569,1.0,17,17.0,2017,0.072158,B,B,2,629,1,NaN,0,0,A,1.0
5,7633,53,1.0,40.675627,NaN,1.0,21,21.0,1977,0.049637,B,B,34,7759,0,229.0,1,3,B,0.0
6,13329,23,2.0,68.099538,64.843025,1.0,2,17.0,1977,0.075779,B,B,6,1437,3,NaN,0,2,B,1.0
7,5502,32,2.0,48.193897,32.857644,6.0,5,14.0,1972,0.135650,B,B,46,7960,6,350.0,3,11,B,0.0
8,4220,96,3.0,72.277106,45.968758,9.0,17,17.0,1997,0.041116,B,B,53,14892,4,NaN,1,4,B,0.0
9,11538,6,3.0,80.219400,47.660260,9.0,13,17.0,2014,0.243205,B,B,5,1564,0,540.0,0,0,B,1.0


In [39]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 0 to 4999
Data columns (total 20 columns):
Id                5000 non-null int64
DistrictId        5000 non-null int64
Rooms             5000 non-null float64
Square            5000 non-null float64
LifeSquare        3959 non-null float64
KitchenSquare     5000 non-null float64
Floor             5000 non-null int64
HouseFloor        5000 non-null float64
HouseYear         5000 non-null int64
Ecology_1         5000 non-null float64
Ecology_2         5000 non-null object
Ecology_3         5000 non-null object
Social_1          5000 non-null int64
Social_2          5000 non-null int64
Social_3          5000 non-null int64
Healthcare_1      2623 non-null float64
Helthcare_2       5000 non-null int64
Shops_1           5000 non-null int64
Shops_2           5000 non-null object
large_district    5000 non-null float64
dtypes: float64(8), int64(9), object(3)
memory usage: 820.3+ KB


In [40]:
test['Price']=model.predict(test.loc[:, feats])

In [41]:
test.head()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,...,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,large_district,Price
0,725,58,2.0,49.882643,33.432782,6.0,6,14.0,1972,0.310199,...,B,11,2748,1,NaN,0,0,B,0.0,226286.067242
1,15856,74,2.0,69.263183,NaN,1.0,6,1.0,1977,0.075779,...,B,6,1437,3,NaN,0,2,B,0.0,247888.883285
2,5480,190,1.0,15.000000,15.948246,12.0,2,5.0,1909,0.000000,...,B,30,7538,87,4702.0,5,5,B,0.0,219549.031311
3,15664,47,2.0,73.046609,51.940842,9.0,22,22.0,2007,0.101872,...,B,23,4583,3,NaN,3,3,B,0.0,329279.279708
4,14275,27,1.0,47.527111,43.387569,1.0,17,17.0,2017,0.072158,...,B,2,629,1,NaN,0,0,A,1.0,139102.781623


In [42]:
test.loc[:,['Id','Price']]

,Id,Price
0,725,226286.067242
1,15856,247888.883285
2,5480,219549.031311
3,15664,329279.279708
4,14275,139102.781623
5,7633,178370.684914
6,13329,185490.717324
7,5502,208015.874671
8,4220,349190.267864
9,11538,222043.401108


In [43]:
test.loc[:,['Id','Price']].shape

(5000, 2)

In [44]:
test.loc[:,['Id','Price']].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 0 to 4999
Data columns (total 2 columns):
Id       5000 non-null int64
Price    5000 non-null float64
dtypes: float64(1), int64(1)
memory usage: 117.2 KB


In [45]:
test.loc[:,['Id','Price']].to_csv('Course_work/YNesterovich_predictions.csv', index=None)